# I. Import modules

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from math import sqrt
from statsmodels.stats.outliers_influence import variance_inflation_factor
from time import time
from scipy import stats
from statistics import median
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, LeaveOneGroupOut, GroupKFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

pd.set_option("display.max_columns", None)

# II. Data exploration

In [2]:
with open('phytoplankton.csv', mode='r') as file:
    df = pd.read_csv(file)
    
print('There are' + '\033[1m', df.duplicated().sum(), '\033[0m' + 'duplicates.')
pd.concat([pd.DataFrame(df.dtypes, columns=['data type']).transpose(), \
           pd.DataFrame(df.isnull().sum(), columns=['missing values']).transpose(), \
           pd.DataFrame(df.describe().loc['count']).transpose(),\
           df.head(n=5)])

There are 0 duplicates.


,Unnamed: 0,replicate,timepoint,Area_M01,Area_M06,Aspect Ratio_M01,Aspect Ratio_M06,Aspect Ratio Intensity_M01_BF,Aspect Ratio Intensity_M06_SSC,Modulation_M01_BF,Modulation_M06_SSC,Contrast_M01_BF,Contrast_M06_SSC,Gradient RMS_M01_BF,Gradient RMS_M06_SSC,Mean Pixel_M01_BF,Mean Pixel_M06_SSC,Median Pixel_M01_BF,Median Pixel_M06_SSC,Length_M01,Length_M06,Width_M01,Width_M06,Height_M01,Height_M06,Saturation Count_M01_BF,Saturation Count_M06_SSC,Saturation Percent_M01_BF,Saturation Percent_M06_SSC,Similarity_M01_Ch02_Ch03,output_viral,output_psba
data type,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
missing values,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
count,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496,290496
0,4,1,0,23.3333,8.44444,0.88908,0.723376,0.81477,0.647845,0.0857143,0.266667,1.4309,42.8254,58.3867,37.2124,-0.0369048,14.731,-6.33214,13.2179,5.66667,3.66667,5,2.66667,5.66667,3.66667,0,0,0,0,0.733362,-1263.33,51540.9
1,15,1,0,26.4444,14.2222,0.952983,0.533022,0.827753,0.524853,0.101124,0.634286,1.99703,267.993,60.9043,43.3637,0.541885,29.15,-3.60938,15.8688,5.66667,6,5.66667,3.33333,5.66667,6,0,0,0,0,1.86956,-1757.93,52686.7
2,18,1,0,30,16.3333,0.920297,0.552871,0.874341,0.328934,0.10987,0.610063,1.79933,274.141,56.7948,48.2439,1.58654,23.4964,-3.2468,13.8365,6.33333,6,6,3.66667,6.33333,6,0,0,0,0,1.05397,-2387.08,46909.6
3,23,1,0,33.4444,22.5556,0.970458,0.558474,0.8494,0.549093,0.089196,0.590062,1.82124,295.625,60.0009,44.0879,1.50062,28.3466,-2.99107,19.9673,6.33333,7,6.33333,4,6.33333,7,0,0,0,0,1.58101,-2913.64,71265.9
4,25,1,0,21.3333,11.2222,0.917762,0.800643,0.817675,0.722305,0.0875866,0.369565,2.38026,129.256,63.8728,43.537,1.27027,12.6424,-4.22973,10.9493,5.33333,4,5,3.33333,5.33333,4,0,0,0,0,1.21549,-1059.5,26822.2


# III. Descriptive statistics
## 1. Univariate
### 1.1 Summary

In [ ]:
pd.concat([df.describe().drop('count'),\
           pd.DataFrame(df.skew(), columns=['skewness']).transpose(),\
           pd.DataFrame(df.kurtosis(), columns=['kurtosis']).transpose()]).round(2)

### 1.2 Plots

In [ ]:
tic = time()

for column in df:
    f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)})
    sns.boxplot(data=df, x=column, ax=ax_box)
    sns.histplot(data=df, x=column, kde=True, ax=ax_hist)
    ax_box.set(xlabel='')
    plt.show()
    
toc = time()
print(f"Done in {toc - tic:.3f}s")

## 2. Bivariate
### 2.1 Correlation

In [ ]:
display(df.corr().round(2))

fig, heat = plt.subplots(figsize=(20,10))
heat = sns.heatmap(df.corr(), annot=True)
plt.show()

### 2.2 Plots

In [ ]:
Z = df[['timepoint', 'Mean Pixel_M01_BF', 'Mean Pixel_M06_SSC', 'Similarity_M01_Ch02_Ch03',\
        'Modulation_M06_SSC', 'Area_M01', 'Width_M01', 'output_viral', 'output_psba']]

tic = time()

g = sns.PairGrid(Z)
g.map_upper(sns.scatterplot)
g.map_lower(sns.kdeplot, cmap = "Blues_d")
g.map_diag(sns.histplot, kde=True)
plt.show()

toc = time()
print(f"Done in {toc - tic:.3f}s")

In [ ]:
tic = time()

for column in df:
    fig, ax = plt.subplots(nrows=1, ncols=2)
    plt.subplot(1, 2, 1)
    sns.scatterplot(data=df, x=column, y="output_viral")
    plt.xlabel(column)
    plt.ylabel("output_viral")
    plt.subplot(1, 2, 2)
    sns.scatterplot(data=df, x=column, y="output_psba")
    plt.xlabel(column)
    plt.ylabel("output_psba")
    plt.show()
    
toc = time()
print(f"Done in {toc - tic:.3f}s")

# IV. Data preparation

In [24]:
# Find and remove outlier
print(np.where(df['output_viral']>100000))
df = df.drop([3230])

# Assign features and targets
X = df.drop(['Unnamed: 0', 'Saturation Count_M01_BF', 'Saturation Percent_M01_BF', 'output_viral', 'output_psba'], axis = 1)
Y_viral = df['output_viral']
Y_psba = df['output_psba']
Replicates = df['replicate']

# Smaller subset for computationally demanding methods
DF = df.sample(n=50000)
x = DF.drop(['Unnamed: 0', 'output_viral', 'output_psba'], axis = 1)
y_viral = DF['output_viral']
y_psba = DF['output_psba']
replicates = DF['replicate']

print('The dimensions of the ' + '\033[4m' + 'features' + '\033[0m' + ' are:', np.shape(X))
print('The dimensions of the target ' + '\033[4m' + 'viral load' + '\033[0m' + ' are:', np.shape(Y_viral))
print('The dimensions of the target ' + '\033[4m' + 'PSBA mRNA' + '\033[0m' + ' are:', np.shape(Y_psba))

(array([], dtype=int64),)
The dimensions of the features are: (290495, 27)
The dimensions of the target viral load are: (290495,)
The dimensions of the target PSBA mRNA are: (290495,)


# V. Model building
## 1. Linear regression 
### 1.1 Function

In [27]:
def LinReg(X, Y, method=LinearRegression, coef='Yes', Replicates=Replicates):
    tic = time()
    
    # Initialize
    count=0
    performance = pd.DataFrame(index=['Mean absolute error', 'Mean squared error', \
                                      'Root mean squared error', 'R\u00b2 (%)', \
                                      'Adjusted R\u00b2 (%)', 'Mean absolute percentage error (%)'])
    if coef=='Yes':
        coefficients = pd.DataFrame(index = np.insert(X.columns, 0, 'Intercept'))

    # Split and standardize data
    logo = LeaveOneGroupOut()
    for train_index, test_index in logo.split(X, Y, Replicates):
        count+=1
        X_train, X_test = X.to_numpy()[train_index], X.to_numpy()[test_index]
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        Y_train, Y_test = Y.to_numpy()[train_index], Y.to_numpy()[test_index]
        Replicates_train, Replicates_test = Replicates.to_numpy()[train_index],\
                                            Replicates.to_numpy()[test_index]
        
        # Optimized hyperparameters
        if method is LinearRegression:
            Model = LinearRegression()
        elif method is Ridge or method is Lasso:
            hyperparameters = {"alpha": 10**np.linspace(10,-2,10)*0.5} 
            gkf=GroupKFold(n_splits=2)
            Grid = GridSearchCV(method(), hyperparameters, scoring='r2', cv=gkf, verbose=0, n_jobs=-1)
            Grid.fit(X_train_scaled, Y_train, Replicates_train)
            Model = method(alpha = Grid.best_params_['alpha'])
        elif method is ElasticNet:
            hyperparameters = {"alpha": 10**np.linspace(10,-2,10)*0.5, \
                               "l1_ratio": np.arange(0, 1, 0.1)}
            gkf=GroupKFold(n_splits=2)
            Grid = GridSearchCV(method(), hyperparameters, scoring='r2', cv=gkf, verbose=3, n_jobs=-1)
            Grid.fit(X_train_scaled, Y_train, Replicates_train)
            Model = method(alpha=Grid.best_params_['alpha'], \
                                 l1_ratio=Grid.best_params_['l1_ratio'])
            
        # Build model and predict
        Model.fit(X_train_scaled, Y_train)
        Y_pred = Model.predict(X_test_scaled)
        residuals = Y_pred - Y_test

        # Coefficients
        if coef=='Yes':
            coefficients[count] = np.insert(Model.coef_, 0, Model.intercept_)
    
        # Performance
        mae = mean_absolute_error(Y_test, Y_pred)
        mse = mean_squared_error(Y_test, Y_pred)
        rmse = np.sqrt(mse)
        R2 = Model.score(X_test_scaled, Y_test)
        adjusted_R2 = 1-((1-R2)*(len(Y_test)-1)/(len(Y_test)-X_test_scaled.shape[1]-1))
        mape = np.mean(np.abs((Y_test - Y_pred) / Y_test))
        performance[count] = [mae, mse, rmse, R2*100, adjusted_R2*100, mape*100] 
        
        # Output
        if method is not LinearRegression:
            print(f"\033[1mλ: \033[0m{np.round(Grid.best_params_['alpha'], 1)}")
        if method is ElasticNet:
            print(f"\033[1mα: \033[0m{Grid.best_params_['l1_ratio']}")
    toc = time()
    print(f"Done in {toc - tic:.3f}s")
    performance_average = pd.DataFrame(performance.mean(axis='columns'), columns=['Mean'])
    performance_average['Std'] = performance.std(axis='columns')
    display(performance_average) 
    if coef=='Yes':
        coefficients_average = pd.DataFrame(coefficients.mean(axis='columns'), columns=['Mean'])
        coefficients_average['Std'] = coefficients.std(axis='columns')
        display(coefficients_average)    
    return residuals, Model

### 1.2 Baseline models with all features
##### A. Performance and coefficients

In [28]:
# Viral
residuals_viral, LinReg_viral= LinReg(X, Y_viral)

Done in 8.200s


,Mean,Std
Mean absolute error,4.607602e+03,3.790402e+02
Mean squared error,4.082820e+07,7.099462e+06
Root mean squared error,6.372669e+03,5.709097e+02
R² (%),2.729120e+01,2.865610e+00
Adjusted R² (%),2.727019e+01,2.867588e+00
Mean absolute percentage error (%),8.883939e+02,5.514519e+02


,Mean,Std
Intercept,3039.915808,313.361263
replicate,-752.657273,364.480795
timepoint,3432.053839,534.037392
Area_M01,135.913044,176.514316
Area_M06,-525.871067,133.255757
Aspect Ratio_M01,472.492886,13.371792
Aspect Ratio_M06,-212.211126,15.462832
Aspect Ratio Intensity_M01_BF,-167.953092,35.214607
Aspect Ratio Intensity_M06_SSC,476.261475,21.438074
Modulation_M01_BF,-1.477018,71.009861


In [ ]:
# PSBA
residuals_psba, LinReg_psba= LinReg(X, Y_psba)

##### B. Diagnostics
###### 1. QQ plot

In [ ]:
fig, (ax_viral, ax_psba) = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
fig = sm.graphics.qqplot(residuals_viral, dist=stats.norm, line='45', fit=True, ax=ax_viral)
ax_viral.set_title('Viral load')
fig = sm.graphics.qqplot(residuals_psba, dist=stats.norm, line='45', fit=True, ax=ax_psba)
ax_psba.set_title('PSBA mRNA')
plt.show()

###### 2. Variance Inflation Factor

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pd.DataFrame([variance_inflation_factor(X_scaled, i) for i in range(len(X.columns))], 
             columns=['VIF'], index = X.columns).round(1)

###### 3. Residual plots and regression line

In [ ]:
tic = time()

for variable in X_test:
    fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(18,5))
    plt.subplot(1, 4, 1)
    sns.residplot(x=X_test[variable], y=Y_viral_test)
    plt.subplot(1, 4, 2)
    sns.regplot(x=X_test[variable], y=Y_viral_test, fit_reg=True, line_kws={'color':'green'})
    plt.subplot(1, 4, 3)
    sns.residplot(x=X_test[variable], y=Y_psba_test)
    plt.subplot(1, 4, 4)
    sns.regplot(x=X_test[variable], y=Y_psba_test, fit_reg=True, line_kws={'color':'green'})
    plt.show()

toc = time()
print(f"Done in {toc - tic:.3f}s")

### 1.3 Transformed targets
##### A. Transformation

In [ ]:
tic = time()

Y_viral_positive = Y_viral + abs(np.min(Y_viral)) + 0.001
Y_psba_positive = Y_psba + abs(np.min(Y_psba)) + 0.001                                         

# Log
Y_viral_log = np.log(Y_viral_positive)
Y_psba_log = np.log(Y_psba_positive)                                       

# Square root
Y_viral_sqrt = np.sqrt(Y_viral_positive)
Y_psba_sqrt = np.sqrt(Y_psba_positive)

# Cubic root
Y_viral_cbrt = np.cbrt(Y_viral_positive)
Y_psba_cbrt = np.cbrt(Y_psba_positive)

toc = time()
print(f"Done in {toc - tic:.3f}s")

In [ ]:
moments = [[Y_viral.skew(), Y_viral.kurtosis()],\
           [Y_viral_log.skew(), Y_viral_log.kurtosis()],\
           [Y_viral_sqrt.skew(), Y_viral_sqrt.kurtosis()],\
           [Y_viral_cbrt.skew(), Y_viral_cbrt.kurtosis()],\
           [Y_psba.skew(), Y_psba.kurtosis()],\
           [Y_psba_log.skew(), Y_psba_log.kurtosis()],\
           [Y_psba_sqrt.skew(), Y_psba_sqrt.kurtosis()],\
           [Y_psba_cbrt.skew(), Y_psba_cbrt.kurtosis()]]
index = ['Viral untransformed', 'Viral log', 'Viral Sqrt', 'Viral Cbrt', 'PSBA untransformed', 'PSBA log', 'PSBA sqrt', 'PSBA cbrt']
skewkurt = pd.DataFrame(moments, columns =['Skewness', 'Kurtosis'], index=index)
display(skewkurt)

In [ ]:
# Histograms of transformed targets
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(16,3.5))
plt.suptitle("Viral load", fontsize=15)
plt.subplot(1, 4, 1)
sns.histplot(x=Y_viral, kde=True)
plt.title("Untransformed", fontsize=10)
plt.ylabel('')
plt.subplot(1, 4, 2)
sns.histplot(x=Y_viral_log, kde=True)
plt.title("Log", fontsize=10)
plt.ylabel('')
plt.subplot(1, 4, 3)
sns.histplot(x=Y_viral_sqrt, kde=True)
plt.title("Square root", fontsize=10)
plt.ylabel('')
plt.subplot(1, 4, 4)
sns.histplot(x=Y_viral_cbrt, kde=True)
plt.title("Cubic root", fontsize=10)
plt.ylabel('')
plt.show()

fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(16,3.5))
plt.suptitle("PSBA mRNA", fontsize=15)
plt.subplot(1, 4, 1)
sns.histplot(x=Y_psba, kde=True)
plt.title("Untransformed", fontsize=10)
plt.ylabel('')
plt.subplot(1, 4, 2)
sns.histplot(x=Y_psba_log, kde=True)
plt.title("Log", fontsize=10)
plt.ylabel('')
plt.subplot(1, 4, 3)
sns.histplot(x=Y_psba_sqrt, kde=True)
plt.title("Square root", fontsize=10)
plt.ylabel('')
plt.subplot(1, 4, 4)
sns.histplot(x=Y_psba_cbrt, kde=True)
plt.title("Cubic root", fontsize=10)
plt.ylabel('')
plt.show()

##### B. Performance

In [ ]:
# Viral
print( '\033[1m' + '\033[4m' + 'Untransformed targets' + '\033[0m')
residuals_viral, LinReg_viral = LinReg(X, Y_viral, coef='No')

print('\033[1m' + '\033[4m' + 'Log transformed targets' + '\033[0m')
residuals_viral_log, LinReg_viral_log = LinReg(X, Y_viral_log, coef='No')

print('\033[1m' + '\033[4m' + 'Square root transformed targets' + '\033[0m')
residuals_viral_sqrt, LinReg_viral_sqrt = LinReg(X, Y_viral_sqrt, coef='No')

print('\033[1m' + '\033[4m' + 'Cubic root transformed targets' + '\033[0m')
residuals_viral_cbrt, LinReg_viral_cbrt = LinReg(X, Y_viral_cbrt, coef='No')

In [ ]:
# PSBA
print( '\033[1m' + '\033[4m' + 'Untransformed targets' + '\033[0m')
residuals_psba, LinReg_psba = LinReg(X, Y_psba, coef='No')

print('\033[1m' + '\033[4m' + 'Log transformed targets' + '\033[0m')
residuals_psba_log, LinReg_psba_log = LinReg(X, Y_psba_log, coef='No')

print('\033[1m' + '\033[4m' + 'Square root transformed targets' + '\033[0m')
residuals_psba_sqrt, LinReg_psba_sqrt = LinReg(X, Y_psba_sqrt, coef='No')

print('\033[1m' + '\033[4m' + 'Cubic root transformed targets' + '\033[0m')
residuals_psba_cbrt, LinReg_psba_cbrt = LinReg(X, Y_psba_cbrt, coef='No')

##### C. Diagnostics
###### 1. QQ plots

In [ ]:
fig, (ax_untransformed, ax_log, ax_sqrt, ax_cbrt) = plt.subplots(nrows=1, ncols=4, figsize=(16,3.5))
plt.suptitle('QQ plot Viral load')
fig = sm.graphics.qqplot(residuals_viral, dist=stats.norm, line='45', fit=True, ax=ax_untransformed)
ax_untransformed.set_title('Untransformed')
fig = sm.graphics.qqplot(residuals_viral_log, dist=stats.norm, line='45', fit=True, ax=ax_log)
ax_log.set_title('Log')
fig = sm.graphics.qqplot(residuals_viral_sqrt, dist=stats.norm, line='45', fit=True, ax=ax_sqrt)
ax_sqrt.set_title('Square root')
fig = sm.graphics.qqplot(residuals_viral_cbrt, dist=stats.norm, line='45', fit=True, ax=ax_cbrt)
ax_cbrt.set_title('Cubic root')
plt.show()

fig, (ax_untransformed, ax_log, ax_sqrt, ax_cbrt) = plt.subplots(nrows=1, ncols=4, figsize=(16,3.5))
plt.suptitle('QQ plot PSBA mRNA')
fig = sm.graphics.qqplot(residuals_psba, dist=stats.norm, line='45', fit=True, ax=ax_untransformed)
ax_untransformed.set_title('Untransformed')
fig = sm.graphics.qqplot(residuals_psba_log, dist=stats.norm, line='45', fit=True, ax=ax_log)
ax_log.set_title('Log')
fig = sm.graphics.qqplot(residuals_psba_sqrt, dist=stats.norm, line='45', fit=True, ax=ax_sqrt)
ax_sqrt.set_title('Square root')
fig = sm.graphics.qqplot(residuals_psba_cbrt, dist=stats.norm, line='45', fit=True, ax=ax_cbrt)
ax_cbrt.set_title('Cubic root')
plt.show()

### 1.4 Dimension reduction
#### 1.4.1 Regularization
#### 1. Ridge
##### A. Performance and coefficients

In [29]:
# Untransformed targets Viral
residuals_viral_RidgeReg, Model_viral_RidgeReg = LinReg(X, Y_viral, method=Ridge)

C:\Users\ThinkPad\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass groups=[2 2 2 ... 3 3 3] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


λ: 23207.9


C:\Users\ThinkPad\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass groups=[1 1 1 ... 3 3 3] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


λ: 23207.9


C:\Users\ThinkPad\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass groups=[1 1 1 ... 2 2 2] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


λ: 1077.2
Done in 20.220s


,Mean,Std
Mean absolute error,4.620577e+03,3.575220e+02
Mean squared error,4.135348e+07,7.280923e+06
Root mean squared error,6.413200e+03,5.800936e+02
R² (%),2.637847e+01,2.540491e+00
Adjusted R² (%),2.635724e+01,2.540571e+00
Mean absolute percentage error (%),8.585545e+02,5.652528e+02


,Mean,Std
Intercept,3039.915808,313.361263
replicate,-634.794484,296.338495
timepoint,3078.325069,797.971920
Area_M01,-19.543927,130.946660
Area_M06,-305.611117,218.011592
Aspect Ratio_M01,204.824888,184.768747
Aspect Ratio_M06,-108.328132,85.471645
Aspect Ratio Intensity_M01_BF,-132.716766,7.784395
Aspect Ratio Intensity_M06_SSC,392.113213,55.873647
Modulation_M01_BF,0.595073,73.033138


In [ ]:
# Untransformed targets PSBA
residuals_psba_RidgeReg, Model_psba_RidgeReg = LinReg(X, Y_psba, method=Ridge)

In [ ]:
# Cubic root transformed targets Viral
residuals_viral_RidgeReg_cbrt, Model_viral_RidgeReg_cbrt = LinReg(X, Y_viral_cbrt, method=Ridge, coef='No')

In [ ]:
# Cubic root transformed targets PSBA
residuals_psba_RidgeReg_cbrt, Model_psba_RidgeReg_cbrt = LinReg(X, Y_psba_cbrt, method=Ridge, coef='No')

##### B. Diagnostics
###### 1. QQ plots

In [ ]:
fig, (ax_viral, ax_psba) = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
fig = sm.graphics.qqplot(residuals_viral_RidgeReg, dist=stats.norm, line='45', fit=True, ax=ax_viral)
ax_viral.set_title('Viral load untransformed')
fig = sm.graphics.qqplot(residuals_viral_RidgeReg_cbrt, dist=stats.norm, line='45', fit=True, ax=ax_psba)
ax_psba.set_title('Viral load cubic root')
plt.show()

In [ ]:
fig, (ax_viral, ax_psba) = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
fig = sm.graphics.qqplot(residuals_psba_RidgeReg, dist=stats.norm, line='45', fit=True, ax=ax_viral)
ax_viral.set_title('PSBA mRNA untransformed')
fig = sm.graphics.qqplot(residuals_psba_RidgeReg_cbrt, dist=stats.norm, line='45', fit=True, ax=ax_psba)
ax_psba.set_title('PSBA mRNA cubic root')
plt.show()

#### 2. Lasso
##### A. Performance and coefficients

In [ ]:
# Untransformed targets Viral
residuals_viral_LassoReg, LassoReg_viral = LinReg(X, Y_viral, method=Lasso)

In [ ]:
# Untransformed targets PSBA
residuals_psba_LassoReg, LassoReg_psba = LinReg(X, Y_psba, method=Lasso)

In [ ]:
# Cubic root transformed targets Viral
residuals_viral_LassoReg_cbrt, LassoReg_viral_cbrt = LinReg(X, Y_viral_cbrt, method=Lasso, coef='No')

In [ ]:
# Cubic root transformed targets PSBA
residuals_psba_LassoReg_cbrt, LassoReg_psba_cbrt = LinReg(X, Y_psba_cbrt, method=Lasso, coef='No')

##### B. Diagnostics
###### 1. QQ plots

In [ ]:
fig, (ax_viral, ax_psba) = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
fig = sm.graphics.qqplot(residuals_viral_RidgeReg, dist=stats.norm, line='45', fit=True, ax=ax_viral)
ax_viral.set_title('Viral load untransformed')
fig = sm.graphics.qqplot(residuals_viral_RidgeReg_cbrt, dist=stats.norm, line='45', fit=True, ax=ax_psba)
ax_psba.set_title('Viral load cubic root')
plt.show()

In [ ]:
fig, (ax_viral, ax_psba) = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
fig = sm.graphics.qqplot(residuals_psba_LassoReg, dist=stats.norm, line='45', fit=True, ax=ax_viral)
ax_viral.set_title('PSBA mRNA untransformed')
fig = sm.graphics.qqplot(residuals_psba_LassoReg_cbrt, dist=stats.norm, line='45', fit=True, ax=ax_psba)
ax_psba.set_title('PSBA mRNA cubic root')
plt.show()

#### 3. Elastic net
##### A. Performance and coefficients

In [ ]:
# Untransformed targets Viral
residuals_viral_ElNetReg, ElNetReg_viral = LinReg(X, Y_viral, method=ElasticNet)

In [ ]:
# Cubic root transformed targets Viral
residuals_viral_ElNetReg_cbrt, ElNetReg_viral_cbrt = LinReg(X, Y_viral_cbrt, method=ElasticNet, coef='No')

In [ ]:
# Untransformed targets PSBA
residuals_psba_ElNetRegReg, ElNetReg_psba = LinReg(X, Y_psba, method=ElasticNet)

In [ ]:
# Cubic root transformed targets PSBA
residuals_psba_ElNetReg_cbrt, ElNetReg_psba_cbrt = LinReg(X, Y_psba_cbrt, method=ElasticNet, coef='No')

##### B. Diagnostics
###### 1. QQ plot

In [ ]:
# Viral
fig, (ax_viral, ax_psba) = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
fig = sm.graphics.qqplot(residuals_viral_ElNetReg, dist=stats.norm, line='45', fit=True, ax=ax_viral)
ax_viral.set_title('Viral load untransformed')
fig = sm.graphics.qqplot(residuals_viral_ElNetReg_cbrt, dist=stats.norm, line='45', fit=True, ax=ax_psba)
ax_psba.set_title('Viral load cubic root')
plt.show()

In [ ]:
# PSBA
fig, (ax_viral, ax_psba) = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
fig = sm.graphics.qqplot(residuals_psba_ElNetReg, dist=stats.norm, line='45', fit=True, ax=ax_viral)
ax_viral.set_title('Viral load untransformed')
fig = sm.graphics.qqplot(residuals_psba_ElNetReg_cbrt, dist=stats.norm, line='45', fit=True, ax=ax_psba)
ax_psba.set_title('Viral load cubic root')
plt.show()

#### 1.4.2 Principal component analysis

In [ ]:
# Viral
for i in range(8, len(X.columns)):
    pca = PCA(n_components=i)
    X_PCA = pd.DataFrame(pca.fit_transform(X))
    print(f"Variance explained per component:{pca.explained_variance_ratio_.round(2)}")
    print(f"\033[1m{i}\033[0m components explain \033[1m{np.round(sum(pca.explained_variance_ratio_)*100)}%\033[0m of the variance.")
    residuals_viral_PCA, Model_viral_PCA = LinReg(X_PCA, Y_viral, coef='No')

In [ ]:
# PSBA
for i in range(8, len(X.columns)):
    pca = PCA(n_components=i)
    X_PCA = pd.DataFrame(pca.fit_transform(X))
    print(f"Variance explained per component:{pca.explained_variance_ratio_.round(2)}")
    print(f"\033[1m{i}\033[0m components explain \033[1m{np.round(sum(pca.explained_variance_ratio_)*100)}%\033[0m of the variance.")
    residuals_psba_PCA, Model_psba_PCA = LinReg(X_PCA, Y_psba, coef='No')

## 2. Random forest tree
### 2.1 Function

In [ ]:
def RanForReg(X, Y, optimize='No'):
    tic = time()
    
    # Initialize
    count=0
    performance = pd.DataFrame(index=['Mean absolute error', 'Mean squared error', \
                                      'Root mean squared error', 'R\u00b2 (%)', \
                                      'Adjusted R\u00b2 (%)', 'Mean absolute percentage error (%)'])

    # Split data
    logo = LeaveOneGroupOut()
    for train_index, test_index in logo.split(X, Y, Replicates):
        count+=1
        X_train, X_test = X.to_numpy()[train_index], X.to_numpy()[test_index]
        Y_train, Y_test = Y.to_numpy()[train_index], Y.to_numpy()[test_index]
        Replicates_train, Replicates_test = Replicates.to_numpy()[train_index],\
                                            Replicates.to_numpy()[test_index]
        
        # Optimize hyperparameters
        if optimize != 'No':
            ## n_estimators: 5, 10, 15, 20
            ## max_depth: 5, 10, 13, 15, 17, 20
            ## max_features: 5, 10, 15, 18, 20
            hyperparameters = {"n_estimators": [15, 20],
                               "max_depth": [13, 15, 17],
                               "max_features": [15, 18, 20]}
            gkf = GroupKFold(n_splits=2)
            Grid = GridSearchCV(RandomForestRegressor(), hyperparameters, scoring='r2', cv=gkf, verbose=3, n_jobs=-1)
            Grid.fit(X_train, Y_train, Replicates_train)
            n_estimators = Grid.best_params_['n_estimators']
            max_depth = Grid.best_params_['max_depth']
            max_features = Grid.best_params_['max_features']
            
        # Set hyperparameters manually
        else:
            n_estimators = 20
            max_depth = 15
            max_features = 13

        # Build model   
        Model = RandomForestRegressor(n_estimators = n_estimators, 
                                      max_depth = max_depth,
                                      max_features = max_features, 
                                      random_state = 13)
        Model.fit(X_train, Y_train)
        Y_pred = Model.predict(X_test)

        # Performance
        mae = mean_absolute_error(Y_test, Y_pred)
        mse = mean_squared_error(Y_test, Y_pred)
        rmse = np.sqrt(mse)
        R2 = Model.score(X_test, Y_test)
        adjusted_R2 = 1-((1-R2)*(len(Y_test)-1)/(len(Y_test)-X_test.shape[1]-1))
        mape = np.mean(np.abs((Y_test - Y_pred) / Y_test))
        performance[count] = [mae, mse, rmse, R2*100, adjusted_R2*100, mape*100] 
    
    # Output
        print(f"\033[1mNumber of trees: \033[0m{n_estimators}")
        print(f"\033[1mMaximum depth: \033[0m{max_depth}")
        print(f"\033[1mMaximum features per node split: \033[0m{max_features}")
    toc = time()
    print(f"Done in {toc - tic:.3f}s")
    performance_average = pd.DataFrame(performance.mean(axis='columns'), columns=['Mean'])
    performance_average['Std'] = performance.std(axis='columns')
    display(performance_average)
    return Model

### 2.2 Random forest models with all features

In [ ]:
# Viral
RanFor_viral = RanForReg(X, Y_viral, optimize='Yes')

In [ ]:
# PSBA
RanFor_psba = RanForReg(X, Y_psba, optimize='Yes')

### 2.3 Dimension reduction
#### 2.3.1 Select n most important features

In [ ]:
# Importance of top n features
n = 15
features = X.columns
importances_viral = RanFor_viral.feature_importances_
importances_psba = RanFor_psba.feature_importances_
indices_viral = np.argsort(importances_viral)[-n:]
indices_psba = np.argsort(importances_psba)[-n:]

# Plot
plt.subplots(nrows=1, ncols=2, figsize=(17,7))
plt.suptitle("Feature's relative importances", fontsize=15)

plt.subplot(1, 2, 1)
plt.title("Viral load", fontsize=12)
plt.barh(range(len(indices_viral)), importances_viral[indices_viral], color='b', align='center')
plt.yticks(range(len(indices_viral)), [features[i] for i in indices_viral])

plt.subplot(1, 2, 2)
plt.title("PSBA mRNA", fontsize=12)
plt.barh(range(len(indices_psba)), importances_psba[indices_psba], color='b', align='center')
plt.yticks(range(len(indices_psba)), [features[i] for i in indices_psba])

plt.show()

In [ ]:
# Select n most important features from model
threshold_viral = np.sort(importances_viral)[-n-1] + 0.00001
threshold_psba = np.sort(importances_psba)[-n-1] + 0.00001
sfm_viral = SelectFromModel(RanFor_viral, threshold=threshold_viral).fit(X_train, Y_viral_train)
sfm_psba = SelectFromModel(RanFor_psba, threshold=threshold_psba).fit(X_train, Y_psba_train)

print(f"\033[1mViral load {n} selected features: \033[0m", features[sfm_viral.get_support()])
print(f"\033[1mPSBA mRNA {n} selected features: \033[0m", features[sfm_psba.get_support()])
X_viral_selected = df[features[sfm_viral.get_support()]]
X_psba_selected = df[features[sfm_psba.get_support()]]

In [ ]:
# Viral
RanFor_viral = RanForReg(X_viral_selected, Y_viral, optimize='No')

In [ ]:
# PSBA
RanFor_psba = RanForReg(X_psba_selected, Y_psba, optimize='No')

#### 2.3.2 Principal Component Analysis

In [ ]:
# Viral
for i in range(5, len(X.columns)):
    pca = PCA(n_components=i)
    X_PCA = pd.DataFrame(pca.fit_transform(X))
    print(f"Variance explained per component:{pca.explained_variance_ratio_.round(2)}")
    print(f"\033[1m{i}\033[0m components explain \033[1m{np.round(sum(pca.explained_variance_ratio_)*100)}%\033[0m of the variance.")
    RanFor_viral_PCA = RanForReg(X_PCA, Y_viral, optimize='No')

In [ ]:
# PSBA
for i in range(5, len(X.columns)):
    pca = PCA(n_components=i)
    X_PCA = pd.DataFrame(pca.fit_transform(X))
    print(f"Variance explained per component:{pca.explained_variance_ratio_.round(2)}")
    print(f"\033[1m{i}\033[0m components explain \033[1m{sum(pca.explained_variance_ratio_)*100}%\033[0m of the variance.")
    RanFor_psba_PCA = RanForReg(X_PCA, Y_psba, optimize='No')

## 3. Multilayer perceptron
### 3.1 Function

In [20]:
def MLPreg(X, Y, Replicates=Replicates):
    tic = time()
    
    # Initialize
    count=0
    performance = pd.DataFrame(index=['Mean absolute error', 'Mean squared error', \
                                      'Root mean squared error', 'R\u00b2 (%)', \
                                      'Adjusted R\u00b2 (%)', 'Mean absolute percentage error (%)'])
        
    # Split and standardize data
    logo = LeaveOneGroupOut()
    for train_index, test_index in logo.split(X, Y, Replicates):
        count+=1
        X_train, X_test = X.to_numpy()[train_index], X.to_numpy()[test_index]
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        Y_train, Y_test = Y.to_numpy()[train_index], Y.to_numpy()[test_index]
        Replicates_train, Replicates_test = Replicates.to_numpy()[train_index],\
                                            Replicates.to_numpy()[test_index]
        
        # Build model and predict
        ## hidden_layer_sizes: (28), (28, 28)
        Model = MLPRegressor(hidden_layer_sizes=(28), activation="relu", random_state=13, max_iter=2000)
        Model.fit(X_train, Y_train)
        Y_pred = Model.predict(X_test)

        # Performance
        mae = mean_absolute_error(Y_test, Y_pred)
        mse = mean_squared_error(Y_test, Y_pred)
        rmse = np.sqrt(mse)
        R2 = Model.score(X_test, Y_test)
        adjusted_R2 = 1-((1-R2)*(len(Y_test)-1)/(len(Y_test)-X_test.shape[1]-1))
        mape = np.mean(np.abs((Y_test - Y_pred) / Y_test)) * 100
        performance[count] = [mae, mse, rmse, R2*100, adjusted_R2*100, mape*100] 
        print(mae, mse, rmse, R2, adjusted_R2, mape)
        
    # Output
    toc = time()
    print(f"Done in {toc - tic:.3f}s")
    performance_average = pd.DataFrame(performance.mean(axis='columns'), columns=['Mean'])
    performance_average['Std'] = performance.std(axis='columns')
    display(performance_average)
    return Model

### 3.2 MLP with all features

In [15]:
# Viral
MLPreg(X, Y_viral, Replicates=Replicates)

3445.221293896806 28394117.217697304 5328.613066990069 0.5475738917312435 0.5474741472680102 617.9143802053653
2347.685013303076 16929628.391157392 4114.5629647822125 0.6388476517027025 0.6387377899054107 249.87245015301193
2855.044330060051 21266923.702682335 4611.607496598375 0.6354763310608738 0.6353520380879452 1097.6948943801847
Done in 3452.805s


,Mean,Std
Mean absolute error,2.882650e+03,5.492887e+02
Mean squared error,2.219689e+07,5.788545e+06
Root mean squared error,4.684928e+03,6.103371e+02
R² (%),6.072993e+01,5.175117e+00
Adjusted R² (%),6.071880e+01,5.174141e+00
Mean absolute percentage error (%),6.551606e+04,4.251367e+04


MLPRegressor(hidden_layer_sizes=(28, 28), max_iter=2000, random_state=13)

In [17]:
# PSBA
MLPreg(X, Y_psba, Replicates=Replicates)

16749.605159327803 520054691.3515765 22804.707657665258 0.41665081225538514 0.41652220373667737 505.4343419137909
9743.6000674753 201863799.0265385 14207.878062066076 0.7333865957422774 0.7333054924962042 416.9820202102362
12057.370754132582 264309431.6034329 16257.596120073622 0.6350379573618659 0.6349135149150484 243.84044450629796
Done in 1894.485s


,Mean,Std
Mean absolute error,1.285019e+04,3.569657e+03
Mean squared error,3.287426e+08,1.685974e+08
Root mean squared error,1.775673e+04,4.490202e+03
R² (%),5.950251e+01,1.621146e+01
Adjusted R² (%),5.949137e+01,1.621354e+01
Mean absolute percentage error (%),3.887523e+04,1.330621e+04


MLPRegressor(hidden_layer_sizes=(28, 28), max_iter=2000, random_state=13)

### 3.3 Dimension reduction
#### 3.3.2 Principal component analysis

In [21]:
# Project features
pca = PCA(n_components=15)
X_PCA = pd.DataFrame(pca.fit_transform(X))

In [22]:
# Viral 15 components
MLPreg(X_PCA, Y_viral)

3535.922021016689 33056751.911851063 5749.500144521354 0.4732804156290854 0.4732159088217245 341.50532299335424
2640.0561833487427 18505131.335540384 4301.759097804104 0.6052381373668522 0.6051714320842173 319.68308895123056
3693.587300917867 29863756.24836259 5464.774126014961 0.48812314615188723 0.4880261961057021 1929.543557704654
Done in 3538.387s


,Mean,Std
Mean absolute error,3.289855e+03,5.682373e+02
Mean squared error,2.714188e+07,7.648129e+06
Root mean squared error,5.172011e+03,7.669884e+02
R² (%),5.222139e+01,7.228309e+00
Adjusted R² (%),5.221378e+01,7.228948e+00
Mean absolute percentage error (%),8.635773e+04,9.232183e+04


MLPRegressor(hidden_layer_sizes=28, max_iter=2000, random_state=13)

In [23]:
# PSBA 15 components
MLPreg(X_PCA, Y_psba)

9477.61012293205 202362826.09157494 14225.428854399257 0.7730081235813331 0.7729803241190023 401.9296992318267


C:\Users\ThinkPad\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


9484.0754570859 201445550.64292443 14193.151540194462 0.7339390010072055 0.733894043082401 401.75662339064235
11171.22438145121 244249984.45132363 15628.499110641547 0.6627363136498463 0.6626724355320484 227.76301577147544
Done in 4329.507s


,Mean,Std
Mean absolute error,1.004430e+04,9.759476e+02
Mean squared error,2.160195e+08,2.445266e+07
Root mean squared error,1.468236e+04,8.195396e+02
R² (%),7.232278e+01,5.591078e+00
Adjusted R² (%),7.231823e+01,5.592865e+00
Mean absolute percentage error (%),3.438164e+04,1.005053e+04


MLPRegressor(hidden_layer_sizes=28, max_iter=2000, random_state=13)

## 4. K-nearest neighbor regression
### 4.1 Function

In [4]:
def KNNReg(X, Y, Replicates=Replicates):   
    tic = time()

    # Initialize
    count=0
    performance = pd.DataFrame(index=['Mean absolute error', 'Mean squared error', \
                                      'Root mean squared error', 'R\u00b2 (%)', \
                                      'Adjusted R\u00b2 (%)', 'Mean absolute percentage error (%)'])

    # Split and standardize data
    logo = LeaveOneGroupOut()
    for train_index, test_index in logo.split(X, Y, Replicates):
        count+=1
        X_train, X_test = X.to_numpy()[train_index], X.to_numpy()[test_index]
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        Y_train, Y_test = Y.to_numpy()[train_index], Y.to_numpy()[test_index]
        Replicates_train, Replicates_test = Replicates.to_numpy()[train_index],\
                                            Replicates.to_numpy()[test_index]
        
        # Optimize hyperparameter K
        ## K: 4, 6, 9, 11, 13, 14, 15, 18, 20, 30
        hyperparameters = {'n_neighbors':[14]}
        gkf = GroupKFold(n_splits=2)
        Grid = GridSearchCV(KNeighborsRegressor(), hyperparameters, scoring='r2', cv=gkf, verbose=3, n_jobs=-1)
        Grid.fit(X_train_scaled, Y_train, Replicates_train)

        # Build models and predict
        Model = KNeighborsRegressor(n_neighbors = Grid.best_params_['n_neighbors'])
        Model.fit(X_train_scaled, Y_train)
        Y_pred = Model.predict(X_test_scaled)
        
        # Performance
        mae = mean_absolute_error(Y_test, Y_pred)
        mse = mean_squared_error(Y_test, Y_pred)
        rmse = np.sqrt(mse)
        R2 = Model.score(X_test_scaled, Y_test)
        adjusted_R2 = 1-((1-R2)*(len(Y_test)-1)/(len(Y_test)-X_test_scaled.shape[1]-1))
        mape = np.mean(np.abs((Y_test - Y_pred) / Y_test))
        performance[count] = [mae, mse, rmse, R2*100, adjusted_R2*100, mape*100] 

        # Output
        print(f"\033[1mViral load K: \033[0m{Grid.best_params_['n_neighbors']}")
    toc = time()
    print(f"Done in {toc - tic:.3f}s")
    performance_average = pd.DataFrame(performance.mean(axis='columns'), columns=['Mean'])
    performance_average['Std'] = performance.std(axis='columns')
    display(performance_average)
    return Model

### 4.2 KNN with all features

In [ ]:
# Viral
KNNReg(X, Y_viral, Replicates=Replicates)

In [ ]:
# PSBA
KNNReg(X, Y_psba, Replicates=Replicates)

### 4.3 Dimension reduction
#### 4.3.1 Principal component analysis

In [5]:
# Project features
pca = PCA(n_components=15)
X_PCA = pd.DataFrame(pca.fit_transform(X))

In [ ]:
# Viral
KNNReg(X_PCA, Y_viral, Replicates=Replicates)

In [6]:
# PSBA
KNNReg(X_PCA, Y_psba, Replicates=Replicates)

C:\Users\ThinkPad\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass groups=[2 2 2 ... 3 3 3] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 20.0min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 20.0min finished


Viral load K: 14


C:\Users\ThinkPad\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass groups=[1 1 1 ... 3 3 3] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


...

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 17.5min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 17.5min finished


Viral load K: 14
Done in 10462.212s


,Mean,Std
Mean absolute error,1.073548e+04,7.824623e+02
Mean squared error,2.580000e+08,1.635881e+07
Root mean squared error,1.605692e+04,5.127017e+02
R² (%),6.719982e+01,3.071295e+00
Adjusted R² (%),6.719451e+01,3.072775e+00
Mean absolute percentage error (%),3.027936e+02,8.708017e+01


KNeighborsRegressor(n_neighbors=14)

## 5. Support vector regression
### 5.1 Function 

In [26]:
def SVReg(X, Y, Replicates=Replicates):
    tic = time()
    
    # Initialize
    count=0
    performance = pd.DataFrame(index=['Mean absolute error', 'Mean squared error', \
                                      'Root mean squared error', 'R\u00b2 (%)', \
                                      'Adjusted R\u00b2 (%)', 'Mean absolute percentage error (%)'])

    # Split and standardize data
    logo = LeaveOneGroupOut()
    for train_index, test_index in logo.split(X, Y, Replicates):
        count+=1
        X_train, X_test = X.to_numpy()[train_index], X.to_numpy()[test_index]
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        Y_train, Y_test = Y.to_numpy()[train_index], Y.to_numpy()[test_index]
        Replicates_train, Replicates_test = Replicates.to_numpy()[train_index],\
                                            Replicates.to_numpy()[test_index]

        # Optimize hyperparameters
        ## C: 0.1, 1, 10, 100, 1000, 10000, 50000, 100000
        ## gamma: 1, 0.1, 0.08, 0.05, 0.03, 0.01, , 0.005, 0.001, 0.0001
        ## kernel: rbf, linear, poly
        hyperparameter = {'C': [10000, 50000], 
                          'gamma': [0.03],
                          'kernel': ['rbf']} 
        gkf = GroupKFold(n_splits=2)
        Grid = GridSearchCV(SVR(), hyperparameter, scoring='r2', cv=gkf, verbose=3, n_jobs=-1)
        Grid.fit(X_train_scaled, Y_train, Replicates_train)

        # Build model and predict
        Model = SVR(C=Grid.best_params_['C'],\
                    gamma=Grid.best_params_['gamma'],\
                    kernel=Grid.best_params_['kernel'])
        Model.fit(X_train_scaled, Y_train)
        Y_pred = Model.predict(X_test_scaled)
      
        # Performance
        mae = mean_absolute_error(Y_test, Y_pred)
        mse = mean_squared_error(Y_test, Y_pred)
        rmse = np.sqrt(mse)
        R2 = Model.score(X_test_scaled, Y_test)
        adjusted_R2 = 1-((1-R2)*(len(Y_test)-1)/(len(Y_test)-X_test_scaled.shape[1]-1))
        mape = np.mean(np.abs((Y_test - Y_pred) / Y_test))
        performance[count] = [mae, mse, rmse, R2*100, adjusted_R2*100, mape*100] 
        
        # Output
        print(f"\033[1mViral load C: \033[0m{Grid.best_params_['C']}")
        print(f"\033[1m           γ: \033[0m{Grid.best_params_['gamma']}")
        print(f"\033[1m           kernel: \033[0m{Grid.best_params_['kernel']}")
    toc = time()
    print(f"Done in {toc - tic:.3f}s")
    performance_average = pd.DataFrame(performance.mean(axis='columns'), columns=['Mean'])
    performance_average['Std'] = performance.std(axis='columns')
    display(performance_average) 

### 5.2 Models with all features

In [27]:
# Viral, subset of 10000
#try 50000!
SVReg(x, y_viral, Replicates=replicates)

C:\Users\ThinkPad\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass groups=[2 2 2 ... 3 3 3] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  3.6min finished


Viral load C: 50000
           γ: 0.03
           kernel: rbf
Fitting 2 folds for each of 2 candidates, totalling 4 fits


C:\Users\ThinkPad\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass groups=[1 1 3 ... 3 1 1] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  5.6min finished


Viral load C: 50000
           γ: 0.03
           kernel: rbf
Fitting 2 folds for each of 2 candidates, totalling 4 fits


C:\Users\ThinkPad\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass groups=[2 1 1 ... 1 1 1] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  7.2min finished


Viral load C: 50000
           γ: 0.03
           kernel: rbf
Done in 33814.717s


,Mean,Std
Mean absolute error,3.314000e+03,5.233708e+02
Mean squared error,2.755289e+07,8.079144e+06
Root mean squared error,5.211117e+03,7.718350e+02
R² (%),5.125250e+01,6.531277e+00
Adjusted R² (%),5.116581e+01,6.527331e+00
Mean absolute percentage error (%),3.852243e+02,1.616341e+02


In [ ]:
# PSBA, subset of 10000
SVReg(x, y_psba, Replicates=replicates)

### 5.3 Dimension reduction
#### 5.3.1 Principal component analysis

In [ ]:
# Project features
pca = PCA(n_components=15)
x_PCA = pd.DataFrame(pca.fit_transform(x))

In [ ]:
# Viral, subset of 10000
SVReg(x_PCA, y_viral, Replicates=replicates)

In [ ]:
# PSBA, subset of 10000
SVReg(x_PCA, y_psba, Replicates=replicates)